In [5]:
import pandas as pd
import h5py
import glob
import numpy as np
import sys
import math
import csv

In [6]:
# open excel sheet 
df = pd.read_csv('../data/segmentation_skeleton_info/m3_nodes.csv')

In [7]:
# todo: for each node find its ID from the h5 file
df

,Node ID,X Coord,Y Coord,Z Coord,Coordination Number
0,0,135,415,2.22489,1
1,1,135,399,17.79920,3
2,2,741,421,2.22489,1
3,3,748,423,2.22489,1
4,4,757,443,2.22489,1
5,5,732,445,2.22489,1
6,6,469,766,2.22489,1
7,7,480,777,8.89960,8
8,8,889,52,4.44980,1
9,9,199,95,242.51400,1


In [8]:
# read the h5 file
h5dir = '../data/segmentation_skeleton_info/slices_unflattened_skeleton_m3.h5'
with h5py.File(h5dir, 'r') as hf:
        # i want them sorted. from 0 to max chunk
        print h5dir, 'List of arrays in this file: \n', hf.keys()
        
        for key in hf.keys():
            data = hf.get(key)
            np_data = np.array(data)

h5dir2 = '../data/segmentation_skeleton_info/slices_unflattened_original_m3.h5'
with h5py.File(h5dir2, 'r') as hf:
        # i want them sorted. from 0 to max chunk
        print h5dir2, 'List of arrays in this file: \n', hf.keys()
        
        for key in hf.keys():
            data = hf.get(key)
            np_data2 = np.array(data)


../data/segmentation_skeleton_info/slices_unflattened_skeleton_m3.h5 List of arrays in this file: 
[u'channel10']
../data/segmentation_skeleton_info/slices_unflattened_original_m3.h5 List of arrays in this file: 
[u'channel10']


In [9]:
print np_data.shape

(999, 999, 449)


In [10]:
def get_non_zero(ID0, ID1, ID2, ID3, ID4, ID5, ID6):
    if (ID0):
        return ID0
    if (ID1):
        return ID1    
    if (ID2):
        return ID2    
    if (ID3):
        return ID3    
    if (ID4):
        return ID4
    if (ID5):
        return ID5
    if (ID6):
        return ID6
    return ID0

def getID(x, y, z, data):
    # access the zth slice
    ID0 = data[x][y][z]
    
    ID1 = data[x][y][z - 1]
    ID2 = data[x][y][z + 1]
        
    ID3 = data[x - 1][y][z]
    ID4 = data[x + 1][y][z]
        
    ID5 = data[x][y - 1][z]
    ID6 = data[x][y + 1][z]
        
    ID = get_non_zero(ID0, ID1, ID2, ID3, ID4, ID5, ID6)
    
    return ID

x_max = -sys.maxint - 1
y_max = -sys.maxint - 1
z_max = -sys.maxint - 1

x_min = sys.maxint
y_min = sys.maxint
z_min = sys.maxint
 
id_max = -sys.maxint - 1
id_min = sys.maxint

fout = open('../data/segmentation_skeleton_info/m3_nodes_Ids.csv', 'w')
writer = csv.writer(fout, delimiter=',')
new_row = ['Node ID', 'X Coord', 'Y Coord',  'Z Coord', 'Coordination Number', 'Object ID']
writer.writerow(new_row)
count = 0
for index, row in df.iterrows():
    x = int(row['X Coord'])
    y = int(row['Y Coord'])
    z = float(row['Z Coord'])
    z = round(z, 4)
    
    z_index = round(z/2.2249, 0)
    z_index = int(z_index)

    ID = getID(x, y, z_index, np_data)
    if (ID < 1):
        ID = getID(x, y, z_index, np_data2)
        if (ID < 1):
            print count, ": ", x, y, z
            count = count + 1

    if (ID > id_max):
        id_max = ID
    
    if (ID < id_min):
        id_min = ID
        
    new_row = [row['Node ID'], row['X Coord'], row['Y Coord'], z_index, row['Coordination Number'], ID]
    writer.writerow(new_row)
    
    if (x > x_max):
        x_max = x
    if (y > y_max):
        y_max = y
    if (z > z_max):
        z_max = z   
        
    if (x < x_min):
        x_min = x
    if (y < y_min):
        y_min = y
    if (z < z_min):
        z_min = z   
        
print "Max: ", x_max, y_max, z_max, id_max
print "Min: ", x_min, y_min, z_min, id_min
fout.close()

Max:  992 985 981.181 915.0
Min:  9 11 2.2249 5.0


In [11]:
df_nodes_ids = pd.read_csv('../data/segmentation_skeleton_info/m3_nodes_Ids.csv')
df_nodes_ids # I want a hash function on node ID and Objet ID as value
nodes_id_dict = df_nodes_ids.set_index('Node ID')['Object ID'].to_dict()

In [14]:
df_segments = pd.read_csv('../data/segmentation_skeleton_info/m3_.segments.csv') 

In [15]:
df_segments

,Segment ID,Node ID #1,Node ID #2,Point IDs
0,0,0,1,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16"
1,1,2,3,"17,18,19,20,21,22,23,24"
2,2,4,5,"25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,4..."
3,3,6,7,"51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66"
4,4,8,9,"67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,8..."
5,5,10,11,"760,761,762,763,764,765,766,767,768,769,770,77..."
6,6,12,12,"781,782,783,784,785"
7,7,12,13,"786,787,788,789,790,791,792,793,794,795,796,79..."
8,8,7,7,"883,884,885"
9,9,14,15,"886,887,888,889,890,891,892,893,894,895,896,89..."


In [45]:
# iterate through df_segments, and label each segment with an ID
fout = open('../data/segmentation_skeleton_info/m3_segments_Ids.csv', 'w')
writer = csv.writer(fout, delimiter=',')
new_row = ['Segment ID', 'Node ID #1',  'Node ID #2', 'Point IDs', 'Object ID']
writer.writerow(new_row)
pointID_objID = {}
for index, row in df_segments.iterrows():
    node1 = row['Node ID #1']
    node2 = row['Node ID #2']
    
    #if (nodes_id_dict[node1] != nodes_id_dict[node2] ): # for now discard these segments
    #    print node1, node2, nodes_id_dict[node1] , nodes_id_dict[node2]
        #continue
    
    ID = nodes_id_dict[node1]
    new_row = [row['Segment ID'], row['Node ID #1'], row['Node ID #2'], z_index, row['Point IDs'], ID]
    writer.writerow(new_row)
    
    points = row['Point IDs'].split(',')
    if ID == 746 or ID == 745:
        print row
        
    for p in points:
        pointID_objID[p] = ID
        
fout.close()

Segment ID                                                   33
Node ID #1                                                   53
Node ID #2                                                   54
Point IDs     3291,3292,3293,3294,3295,3296,3297,3298,3299,3...
Name: 33, dtype: object
Segment ID                                                   41
Node ID #1                                                   66
Node ID #2                                                   67
Point IDs     3954,3955,3956,3957,3958,3959,3960,3961,3962,3...
Name: 41, dtype: object
Segment ID                                                   99
Node ID #1                                                  142
Node ID #2                                                  143
Point IDs     8187,8188,8189,8190,8191,8192,8193,8194,8195,8...
Name: 99, dtype: object
Segment ID                                                  117
Node ID #1                                                  166
Node ID #2                      

In [32]:
df_segments_ids = pd.read_csv('../data/segmentation_skeleton_info/m3_segments_Ids.csv') 
df_segments_ids


,Segment ID,Node ID #1,Node ID #2,Point IDs,Object ID
0,0,1,441,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16",109.0
1,2,3,441,"17,18,19,20,21,22,23,24",420.0
2,4,5,441,"25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,4...",420.0
3,6,7,441,"51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66",422.0
4,8,9,441,"67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,8...",264.0
5,10,11,441,"760,761,762,763,764,765,766,767,768,769,770,77...",30.0
6,12,12,441,"781,782,783,784,785",335.0
7,12,13,441,"786,787,788,789,790,791,792,793,794,795,796,79...",335.0
8,7,7,441,"883,884,885",422.0
9,14,15,441,"886,887,888,889,890,891,892,893,894,895,896,89...",510.0


In [40]:
# label points!
# open excel sheet 
def getPointID(x):
    return pointID_objID[str(x)]

df_points = pd.read_csv('../data/segmentation_skeleton_info/m3_points.csv')
df_points['Object ID'] = df_points['Point ID'].apply(getPointID)


In [43]:
df_points.to_csv('../data/m3_points_ids.csv',  index=False)